this script creates a final patient list used for Volume rendering project

In [2]:
import os
import numpy as np
import function_list_VR as ff
import pandas as pd

In [34]:
main_data_path = '/Volumes/McVeighLab/wip/zhennong/'
main_info_path = '/Users/zhennongchen/Documents/Zhennong_CT_Data/Patient_Overview'
patient_list = ff.find_all_target_files(['Abnormal/*','Normal/*'],os.path.join(main_data_path,'nii-images'))
normal_info = pd.read_csv(os.path.join(main_info_path,'Case_search_list/normal_candidates.csv'))
abnormal_info = pd.read_csv(os.path.join(main_info_path,'Case_search_list/abnormal_candidates.csv'))
entire_info = [normal_info,abnormal_info]
timeframe_info = pd.read_csv(os.path.join(main_info_path,'Case_search_list/time_frame_assignment_for_pretrained_AI_prediction.csv'))

In [45]:
for count in range(0,len(patient_list)):
    p = patient_list[count]
    patient_id = os.path.basename(p)
    patient_class = os.path.basename(os.path.dirname(p))
    
    
    if patient_class == 'Abnormal':
        info = entire_info[1]
    else:
        info = entire_info[0]
    
    assert (patient_id in info.Patient_ID.values) == True
    assert (patient_id in timeframe_info.Patient_ID.values) == True
    
    t = timeframe_info.loc[timeframe_info['Patient_ID'] == patient_id]
    t = t.iloc[:,1:t.shape[1]]
    
    i = info.loc[info['Patient_ID'] == patient_id]
    i = i.iloc[:,1:i.shape[1]]
    
    if count == 0:
        final_list = pd.merge(t, i, on='Patient_ID')
    else:
        final_list = pd.concat([final_list,pd.merge(t, i, on='Patient_ID')],axis = 0)
   

In [46]:
# add one empty column
final_list.insert(2,'ZC_notes','')
print(final_list.columns.values,final_list.shape)

['Patient_Class' 'Patient_ID' 'ZC_notes' 'ED/ES' 'Timeframe_picked'
 'Total_timeframes' 'Keywords' 'Highlight text' 'Report text' 'Acession'
 'Manufacturer' 'Model' 'Sex' 'Age' 'Protocol'] (375, 15)


In [47]:
final_list.to_excel(os.path.join(main_info_path,'Case_search_list/Final_patient_list.xlsx'),index=True)